# **Sharing All Data Received from Alters**

Considering $U_a = \frac{a+e}{D}$ and $U_e = \frac{a}{a+e}$ result in Pareto-optimal deal:

$U_e \cdot U_a = \frac{a}{D}$

This yields that the optimal deal is only determined by the size of the shared alter data.

In [1]:
import numpy as np
import pandas as pd

import json

from multiprocessing.pool import Pool

from tqdm.notebook import trange

import copy

In [2]:
RESULTS_ROOT = "../01_simulation/04_results/"
SEEDS = [42, 1234, 1867, 613, 1001]
TIME_LIMIT = 300
FALL_BACK_TIME = 15 #seconds -> corresponds to max. 208.3 m

In [3]:
import os


if not(os.path.exists("08_shared_veh_data/alters")):
    os.makedirs("08_shared_veh_data/alters")
for s in SEEDS:
    if not(os.path.exists(f"08_shared_veh_data/alters/{s}")):
        os.makedirs(f"08_shared_veh_data/alters/{s}")

In [4]:
#reading *test* vehicles:

with open("veh_list.json") as f:
    veh_list  = json.load(f)

test_vehicles = veh_list["test_vehs"]

In [5]:
p_data = pd.read_csv("08_shared_veh_data/combined_dataset.csv")
p_data["time"] = p_data["time"].astype(int)
p_data = p_data[p_data["veh_id"].isin(test_vehicles)]
receive_time = [-1]*len(p_data)
p_data["receive_time"] = receive_time

In [6]:
def combine_commuters(veh_id):
    if veh_id.startswith("carIn"):
        return veh_id.split(":")[0]
    return veh_id

#reading moving simulation data:

m_data = pd.DataFrame()
for s in SEEDS:
    filename = f"{RESULTS_ROOT}/vehicle_positions_{s}.csv"
    mf = pd.read_csv(filename)
    mf["seed"] = [s]*len(mf)
    m_data = pd.concat([m_data, mf])

m_data["veh_id"] = m_data["veh_id"].apply(combine_commuters)
m_data = m_data[m_data["veh_id"].isin(test_vehicles)]

In [7]:
with open("edge_maps.json") as f:
    edge_maps = json.load(f)

edge_to_idx = edge_maps["edge_to_idx"]
idx_to_edge = edge_maps["idx_to_edge"]

In [8]:
def collect_meeting_vehicles(m_data, meeting_time, seed, meeting_times, meeting_time_gap=TIME_LIMIT):
    #collecting recently met vehicles:
    
    meetings = m_data[m_data["time"] == meeting_time]
    meetings = meetings[meetings["seed"] == seed]

    mets = []

    #same edges:
    for edge in edge_to_idx:
        vehs = meetings[meetings["edge"] == edge]["veh_id"].unique()
        for i in range(len(vehs)):
            for j in range(i+1, len(vehs)):
                sender = vehs[i]
                receiver = vehs[j]
                #if not met yet or met long time ago:
                if ((not((sender, receiver) in meeting_times)) or
                    meeting_time - meeting_times[(sender, receiver)] > meeting_time_gap):
                    mets.append((sender, receiver))
                    mets.append((receiver, sender)) #they meet vice-versa

    #opposed edges:
    for edge in edge_to_idx:
        #only "reversed" edges are processed, to avoid duplicated meetings:
        if edge.startswith("-"):
            veh_edge = meetings[meetings["edge"] == edge]["veh_id"].unique()
            contra_edge = edge.split("-")[1]
            veh_contra = meetings[meetings["edge"] == contra_edge]["veh_id"].unique()
            for sender in veh_edge:
                for receiver in veh_contra:
                    #if not met yet or met long time ago:
                    if ((not((sender, receiver) in meeting_times)) or
                        meeting_time - meeting_times[(sender, receiver)] > meeting_time_gap):
                        mets.append((sender, receiver))
                        mets.append((receiver, sender)) #they meet vice-versa


    return mets #vehicles at the same time, at the same place, not the 'ego' vehicle and not met recently

In [9]:
def collect_data_upon_meeting(senders_data, meeting_time, seed, time_limit=TIME_LIMIT, ego_time=FALL_BACK_TIME):
    send_data = senders_data[senders_data["seed"] == seed]
    send_data = send_data[send_data["time"] <= meeting_time]
    send_data = send_data[send_data["time"] >= meeting_time-time_limit]

    alters_data = send_data[send_data["receive_time"]>-1]
    has_alters = True if len(alters_data)>0 else False
    alters_data = pd.concat([alters_data, send_data[send_data["time"] >= meeting_time-ego_time]],
                            ignore_index=True)

    return alters_data.drop(columns=["receive_time"]), has_alters

In [10]:
def receive_data(args):
    sender, receiver, time, seed, vehicles_kb = args

    received_data, has_alters = collect_data_upon_meeting(vehicles_kb[sender], time, seed)

    #storing shared data:
    store_shared_data = {
        "sender": sender,
        "receiver": receiver,
        "time": time,
        "data": received_data.to_json(),
        "contains_alter": has_alters
    }

    #fusing data into the receiver vehicle's dataset:
    rec_t = [time]*len(received_data)
    received_data["receive_time"] = rec_t

    updated_kb = pd.concat([vehicles_kb[receiver], received_data], ignore_index=True)
    updated_kb = updated_kb.drop_duplicates(subset="hash", ignore_index=True)

    return store_shared_data, updated_kb

In [11]:
def per_seed_script(seed):
    ps_data = p_data[p_data["seed"] == seed]
    meeting_times = {}
    vehicles_kb = {}
    store_sharing = []

    for veh in test_vehicles:
        vehicles_kb[veh] = copy.deepcopy(ps_data[ps_data["veh_id"] == veh])

    for t in trange(min(ps_data["time"]), max(ps_data["time"])):
        meetings = collect_meeting_vehicles(m_data, t, seed, meeting_times)
        for sender, receiver in meetings:
            arguments = [sender, receiver, t, seed, vehicles_kb]
            meeting_times[(sender, receiver)] = t
            
            new_store, updated_kb = receive_data(arguments)
            vehicles_kb[receiver] = copy.deepcopy(updated_kb)
            store_sharing.append(new_store)

    for veh in vehicles_kb:
        vehicles_kb[veh].to_csv(f"08_shared_veh_data/alters/{seed}/{veh}.csv", index=False)
    store_dict = {
        "shared_data": store_sharing
    }
    with open(f"08_shared_veh_data/alters/{seed}/shared_data.json", "w") as f:
        json.dump(store_dict, f)

with Pool(5) as ps:
    ps.map(per_seed_script, SEEDS)